# Kentucky 3D 

A script that downloads lidar point clouds for any point or area of interest in Kentucky. It will buffer to a selected distance and 
* download and extract lidar LAZ files and NAIP imagery
* create a LAS dataset based on selected lidar class codes
* create a DEM, a colorized point cloud, and raster modeling cliff heights
* create a [Potree](http://potree.org/) 3D map
The output can then be used in ArcGIS Pro for animation and 3D viewing

## Important 💡 

If you want to publish the *potree* folder containing the interactive 3D map to a GitHub.com repository, make sure the *octree.bin* file is less than 100 MB. The file can be found here: `potree\pointclouds\index\octree.bin`.

## Setup

Make sure you run the [get-tools.ipynb](get-tools.ipynb) to download grids and conversion utilities.

## Instructions

Edit the statements labeled with 💡💡💡 for your root GIS folder location and area of interest name, location, and buffer distance. Run the cells top to bottom. Make a copy of this notebook if you want edit the cells labeled 'DO NOT EDIT'.

In [1]:
################ DO NOT EDIT ################ 

# Import ArcGIS package
import arcpy
# Subprocess allows us to issue commands on the command line
import subprocess
# Module to download files with an URL
import urllib.request
# Zip utlity to extract files
from zipfile import ZipFile
# pandas to view table and possibly analyze data in the future?
import pandas as pd
# Date and time modules to print when code was executed
import datetime
def printComplete():
    print("Cell finished processing", datetime.datetime.now())

In [2]:
############### SET LOCAL VARIABLES ###############

# Directory for your GIS projects 
# 💡💡💡
root = "c:\\SarahsGIS"

# Directory name containing your lidar and data tools
tools = "tools"

# Check to see if we've got our tools!
utilities = ['grids\\Kentucky_5k_PointCloudGrid.shp',
             'grids\\Kentucky_10K_NAIP.shp',
             'LAStools\\bin\\laszip64.exe',
             'Potree\\PotreeConverter.exe'
            ]
for utility in utilities:
    check = subprocess.run(f'if exist {root}\\{tools}\\{utility} echo {utility} exists!', shell=True, stdout=subprocess.PIPE)
    print(check.stdout.decode('UTF-8'))

printComplete()

grids\Kentucky_5k_PointCloudGrid.shp exists!

grids\Kentucky_10K_NAIP.shp exists!

LAStools\bin\laszip64.exe exists!

Potree\PotreeConverter.exe exists!

Cell finished processing 2021-05-10 18:41:37.106441


In [3]:
############### SET PROJECT DETAILS ###############

# Project name - creates a folder in GIS directory.
# 💡💡💡
project = "silo1200"

# Point location of the center of the area of interest.
# 💡💡💡
long = -85.533181
lat = 38.116341
# How far in feet from the above point is the area of interest?
# 💡💡💡
buffer_distance = 1000

printComplete()

Cell finished processing 2021-05-10 18:41:42.690024


In [4]:
################ DO NOT EDIT ################ 

# NAIP edition
naip_year = "2018" # 2018, 2016, 2006 (all 2-ft resolution orthophoto)

# lidar folder location which contains the geodatabase of grid layers, lidar file extration utility, and the Potree converter
lidarFolder = f"{root}\\{tools}"

# Output geodatabase name
geodb = "workspace.gdb"

############### Local utilities and grids ###############

# Locations of index grids
las_grid = f"{lidarFolder}\\grids\\Kentucky_5k_PointCloudGrid.shp"
naip_grid = f"{lidarFolder}\\grids\\Kentucky_10K_NAIP.shp"

# Point the script to the directory with the laszip64.exe
las_tools = f"{lidarFolder}\\LAStools\\bin\\laszip64.exe"
las_merge = f"{lidarFolder}\\LAStools\\bin\\lasmerge64.exe"

# Point the script to the directory with the PotreeConverter.exe
potree_tools = f"{lidarFolder}\\Potree\\PotreeConverter.exe"

############### Local download folders ###############

# Downloads folder
downloads = f'{root}\\{project}\\downloads\\'
lidar = f'{root}\\{project}\\lidar\\'
lidar_extract = f'{root}\\{project}\\lidar_extract\\'
lidar_color = f'{root}\\{project}\\lidar_color\\'

# Create folders
print("Making folders...")
folders = [f'{root}\\{project}', downloads, lidar, lidar_extract, lidar_color]
for folder in folders:
    subprocess.run(f'mkdir {folder}', shell=True, stdout=subprocess.PIPE)
    print(f"mkdir {folder}")

printComplete()

Making folders...
mkdir c:\SarahsGIS\silo1200
mkdir c:\SarahsGIS\silo1200\downloads\
mkdir c:\SarahsGIS\silo1200\lidar\
mkdir c:\SarahsGIS\silo1200\lidar_extract\
mkdir c:\SarahsGIS\silo1200\lidar_color\
Cell finished processing 2021-05-10 18:41:48.643963


In [5]:
################ DO NOT EDIT ################ 

############### Create project geodatabase ###############

# LAS dataset name
las_dataset = f'{lidar}\\{project}.lasd'

arcpy.env.overwriteOutput = True
spatial_reference = arcpy.Describe(las_grid).spatialReference

# Create project geodatabase
print(f"Creating {root}\\{project}\\{geodb}")
arcpy.CreateFileGDB_management(f'{root}\\{project}', geodb)

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# Show contents of project directory
completed = subprocess.run(f'dir {root}\\{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

printComplete()

Creating c:\SarahsGIS\silo1200\workspace.gdb
 Volume in drive C is Windows
 Volume Serial Number is 6670-36EA

 Directory of c:\SarahsGIS\silo1200

05/10/2021  06:42 PM    <DIR>          .
05/10/2021  06:42 PM    <DIR>          ..
05/10/2021  06:27 PM    <DIR>          downloads
05/10/2021  06:27 PM    <DIR>          lidar
05/10/2021  06:27 PM    <DIR>          lidar_color
05/10/2021  06:27 PM    <DIR>          lidar_extract
05/10/2021  06:42 PM    <DIR>          workspace.gdb
               0 File(s)              0 bytes
               7 Dir(s)  782,075,281,408 bytes free

Cell finished processing 2021-05-10 18:42:01.288067


In [6]:
################ DO NOT EDIT ################ 

############### Creating area of interest ###############

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# try:
#     f"{root}\\{project}\\{geodb}\\{aoi}"
#     print(f"Using {aoi}")
# except:

print(f"Creating point at lat: {lat}, long: {long}")
# Create a feature class with a spatial reference of GCS WGS 1984
result = arcpy.management.CreateFeatureclass(
    arcpy.env.workspace, 
    "test", "POINT", spatial_reference=4326)
feature_class = result[0]
with arcpy.da.InsertCursor(feature_class, ['SHAPE@']) as cursor:
    cursor.insertRow([(long, lat)])
aoi = "aoi"
arcpy.Project_management("test", f'{root}\\{project}\\{geodb}\\aoi', spatial_reference)
arcpy.Delete_management (f'{root}\\{project}\\{geodb}\\test')

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'
    
# Buffer point
arcpy.Buffer_analysis(aoi, f'{project}_{buffer_distance}ft', buffer_distance)

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# Create a temp layer to find which LAS files to download
arcpy.Intersect_analysis ([f'{project}_{buffer_distance}ft', las_grid], "temp")

layers = arcpy.ListFeatureClasses()
for layer in layers:
    print(f"created {layer}")
    
print(f"Project database: {arcpy.env.workspace}")
printComplete()

Creating point at lat: 38.116341, long: -85.533181
created aoi
created silo1200_1000ft
created temp
Project database: c:\SarahsGIS\silo1200\workspace.gdb
Cell finished processing 2021-05-10 18:42:21.709614


In [7]:
################ DO NOT EDIT ################ 

############### Downloading and extracting LAZ files ###############

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# Find URLs and download them and use laszip64.exe to convert 
cursor = arcpy.da.SearchCursor("temp", ['ftppath', 'LASVersion', 'Year'])
i = 0
las_names = []
for row in cursor:
    url = row[0]
    name = url[-12:]
    las_names.append(f'{lidar}{url[-12:-4]}.las')
    with urllib.request.urlopen(url) as response: 
        print(f'get {url}')
        with open(f'{downloads}{name}', 'wb') as outFile:
            data = response.read()
            outFile.write(data)   
    print(f'{las_tools} -i {downloads}{name} -o {las_names[i]}')
    completed = subprocess.run(f'{las_tools} -i {downloads}{name} -o {las_names[i]}', shell=True, stdout=subprocess.PIPE)
    i += 1

arcpy.CreateLasDataset_management (las_names, las_dataset, "#", "#", spatial_reference, True, True)

arcpy.ExtractLas_3d (las_dataset, lidar_extract, "#", f'{project}_{buffer_distance}ft',  "#", "_extract", "#", "#", True, f'{lidar_extract}{project}_extract.lasd')

arcpy.LasDatasetStatistics_management (f'{lidar_extract}{project}_extract.lasd', "#", f'{root}\\{project}\\stats.csv', "#", "#", "#")
with open(f'{root}\\{project}\\stats.csv', encoding='utf-8') as csv:
    reader = pd.read_csv(csv)
    pdData = pd.DataFrame(reader)

print(f"Project database: {arcpy.env.workspace}")
printComplete()
print('Check your point classification codes...')
pdData[pdData["Category"] == "ClassCodes"]


get ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N086E241.laz
c:\SarahsGIS\tools\LAStools\bin\laszip64.exe -i c:\SarahsGIS\silo1200\downloads\N086E241.laz -o c:\SarahsGIS\silo1200\lidar\N086E241.las
get ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N087E241.laz
c:\SarahsGIS\tools\LAStools\bin\laszip64.exe -i c:\SarahsGIS\silo1200\downloads\N087E241.laz -o c:\SarahsGIS\silo1200\lidar\N087E241.las
Project database: c:\SarahsGIS\silo1200\workspace.gdb
Cell finished processing 2021-05-10 18:43:27.950873
Check your point classification codes...


,Item,Category,Pt_Cnt,Percent,Z_Min,Z_Max,Intensity_Min,Intensity_Max,Synthetic_Pt_Cnt,Range_Min,Range_Max
9,1_Unclassified,ClassCodes,43748.0,8.14,553.87,1264.86,0.0,192.0,0.0,NaN,NaN
10,2_Ground,ClassCodes,239079.0,44.47,529.61,670.18,0.0,231.0,0.0,NaN,NaN
11,3_Low_Vegetation,ClassCodes,79012.0,14.70,531.24,675.12,0.0,196.0,0.0,NaN,NaN
12,4_Medium_Vegetation,ClassCodes,54980.0,10.23,546.33,681.08,0.0,163.0,0.0,NaN,NaN
13,5_High_Vegetation,ClassCodes,109558.0,20.38,544.99,727.18,0.0,158.0,0.0,NaN,NaN
14,6_Building,ClassCodes,2750.0,0.51,624.31,728.97,0.0,65535.0,0.0,NaN,NaN
15,7_Low_Point(noise),ClassCodes,312.0,0.06,552.00,666.73,0.0,175.0,0.0,NaN,NaN
16,8_Model_Key_Point,ClassCodes,8136.0,1.51,531.59,669.90,0.0,209.0,0.0,NaN,NaN


In [8]:
################ DO NOT EDIT ################ 

############### Created bare-earth DEM and hillshade ###############

# default ground classes
# consider adding water to both
las_ground = [2]
las_trees = [1, 2]

# Filter for ground points 
arcpy.MakeLasDatasetLayer_management (f'{lidar_extract}{project}_extract.lasd', f'{lidar}ground', las_ground)
# Filter for ground  and above ground points
arcpy.MakeLasDatasetLayer_management (f'{lidar_extract}{project}_extract.lasd', f'{lidar}trees', las_trees)

print(f"{arcpy.env.workspace}")

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# # Ceate DEM and hillshade
arcpy.LasDatasetToRaster_conversion (f'{lidar}ground', f'{root}\\{project}\\{geodb}\\{project}_dem_5ft', "#", "#", "#", "#", 5)
arcpy.HillShade_3d(f'{project}_dem_5ft', f'{root}\\{project}\\{geodb}\\{project}_hillshade', 270, 55)

print(f"Project database: {arcpy.env.workspace}")
printComplete()

c:\SarahsGIS\silo1200\workspace.gdb
Project database: c:\SarahsGIS\silo1200\workspace.gdb
Cell finished processing 2021-05-10 18:44:07.791860


In [9]:
################ DO NOT EDIT ################ 

############### Downloading and extracting aerial imagery files ###############

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

arcpy.Intersect_analysis ([f'{project}_{buffer_distance}ft', naip_grid], "temp")

# Find URLs, download them and extract 
cursor = arcpy.da.SearchCursor("temp", ['ftppath16', 'TileName'])
i = 0
naip_names = []
if naip_year == "2016":
    # NAIP files have prefix
    naip_prefix = "ky_2ft_naip_2016_"
    extension = "jpg"
elif naip_year == "2018":
    naip_prefix = "KY_2FT_NAIP_2018_"
    extension = "tif"
elif naip_year == "2006":
    naip_prefix = "ky_2ft_naip_2006_"
    extension = "jpg"
for row in cursor:
    url = row[0].replace("_2016_", f"_{naip_year}_")
    if naip_year == "2006":
        name = row[1].lower()
    else:
        name = row[1]
    naip_names.append(name)
    print(f'downloading {url}...')
    with urllib.request.urlopen(url) as response: 
        with open(f'{downloads}{name}.zip', 'wb') as outFile:
            data = response.read()
            outFile.write(data) 
    with ZipFile(f'{downloads}{name}.zip', 'r') as zip: 
        zip.extractall(f'{downloads}{name}')
    print(f'{downloads}{name}\\{naip_prefix}{name}.{extension}')
    arcpy.CopyRaster_management (f'{downloads}{name}\\{naip_prefix}{name}.{extension}', f'{root}\\{project}\\{geodb}\\{name}')
    i += 1
    
print(f"{arcpy.env.workspace}")
# If multiple NAIPs, then mosaic to new raster and clip

arcpy.Delete_management (f'{root}\\{project}\\{geodb}\\temp')
if naip_year == "2018":
    bands = 4
else:
    bands = 3
if len(naip_names) > 1:
    print(f'Mosaic to new raster...')
    arcpy.MosaicToNewRaster_management (naip_names, f'{root}\\{project}\\{geodb}', "tempRaster", None, "8_BIT_UNSIGNED", None, bands)
    arcpy.Clip_management ('tempRaster', '#', f'{root}\\{project}\\{geodb}\\{project}_naip', f'{project}_{buffer_distance}ft', "#", True)
    arcpy.Delete_management (f'{root}\\{project}\\{geodb}\\tempRaster')
else:
    arcpy.Clip_management (naip_names[0], '#', f'{root}\\{project}\\{geodb}\\{project}_naip', f'{project}_{buffer_distance}ft', "#", True)

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'
    

for image in naip_names:
    print(f"Deleting {image}")
    arcpy.Delete_management(f'{root}\\{project}\\{geodb}\\{image}')

print(f" Project database: {arcpy.env.workspace}")
printComplete()


downloading ftp://ftp.kymartian.ky.gov/FSA_NAIP_2018_2FT/ky_2ft_naip_2018_N043E121.zip...
c:\SarahsGIS\silo1200\downloads\N043E121\KY_2FT_NAIP_2018_N043E121.tif
downloading ftp://ftp.kymartian.ky.gov/FSA_NAIP_2018_2FT/ky_2ft_naip_2018_N044E121.zip...
c:\SarahsGIS\silo1200\downloads\N044E121\KY_2FT_NAIP_2018_N044E121.tif
c:\SarahsGIS\silo1200\workspace.gdb
Mosaic to new raster...
Deleting N043E121
Deleting N044E121
 Project database: c:\SarahsGIS\silo1200\workspace.gdb
Cell finished processing 2021-05-10 18:48:11.051558


In [10]:
################ DO NOT EDIT ################ 

############### Creating Colorized point cloud ###############

# Extract LAS points in buffer and colorize
print(f'Creating colorized point cloud at {lidar_color}{project}_rgb.lasd')

arcpy.ColorizeLas_3d (f'{lidar_extract}{project}_extract.lasd', f'{project}_naip', 'RED Band_1; GREEN Band_2; BLUE Band_3', lidar_color, "_color", "#",  "#",  "#",  "#", True, f'{lidar_color}{project}_rgb.lasd')

print(f"Project database: {arcpy.env.workspace}")
printComplete()

Creating colorized point cloud at c:\SarahsGIS\silo1200\lidar_color\silo1200_rgb.lasd
Project database: c:\SarahsGIS\silo1200\workspace.gdb
Cell finished processing 2021-05-10 18:48:35.650931


In [11]:
################ DO NOT EDIT ################ 

############### Surface models ###############

# assign project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# Get elevagtion, which estimates tree heights and some building edges and towers
print(f"Creating {project}_z_range raster measuring the height between the first and last return, an estimation of tree heights.")
arcpy.LasPointStatsAsRaster_management(f'{lidar_color}{project}_rgb.lasd', f"{project}_z_range", "Z_RANGE", "CELLSIZE", 5)

# Create DSM of cliffs over 30 feet in 30-ft diameter neighborhood from bare-earth DEM
print(f'Creating {project}_relief_within_30ft raster measuring range of elevation within 30 feet of each cell.')
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(f'{project}_dem_5ft', neighborhood, "RANGE")
outFocalStat.save(f'{project}_relief_within_30ft')
# Find cliffs over 30 feet
# cliffs_over_30ft = arcpy.sa.Con(outFocalStat > 30, outFocalStat)
# cliffs_over_30ft.save("cliffs_over_30ft")

print(f"Project database: {arcpy.env.workspace}")
printComplete()

Creating silo1200_z_range raster measuring the height between the first and last return, an estimation of tree heights.
Creating silo1200_relief_within_30ft raster measuring range of elevation within 30 feet of each cell.
Project database: c:\SarahsGIS\silo1200\workspace.gdb
Cell finished processing 2021-05-10 18:49:07.381416


In [12]:
################ DO NOT EDIT ################ 

############### Create web 3d point cloud viewer ###############

for i in las_names:
    with open(f'{root}\\{project}\\potree_las_list.txt', 'a+') as outFile:
        a = i.replace("\\lidar\\", "\\lidar_color\\")
        b = a.replace(".las", "_extract_color.las")
        print(b)
        outFile.write(f"{b}\n")

completed = subprocess.run(f"{las_merge} -lof {root}\\{project}\\potree_las_list.txt -o {lidar_color}\\merge.las")
completed = subprocess.run(f"{potree_tools} -i {lidar_color}\\merge.las -o {root}\\{project}\\potree --generate-page index")
print(f"The {root}\\{project}\\potree folder contains an interactive 3D map viewable in a browser.")
print(f"The {lidar_color}\\merge.laz file is a compressesd LAS file of the area of interest to use in open-source applications.")
printComplete()

c:\SarahsGIS\silo1200\lidar_color\N086E241_extract_color.las
c:\SarahsGIS\silo1200\lidar_color\N087E241_extract_color.las
The c:\SarahsGIS\silo1200\potree folder contains an interactive 3D map viewable in a browser.
The c:\SarahsGIS\silo1200\lidar_color\\merge.laz file is a compressesd LAS file of the area of interest to use in open-source applications.
Cell finished processing 2021-05-10 18:50:27.587151


In [13]:
############### Clean up ###############
# !!!!
# This cell will delete the following:
# downloads folder
# lidar folder
# lidar extract folder

# The following will NOT be deleted

    # 1. geodatabase folder
    # 2. lidar_color folder, which contains the colorized point clouds

deleteThis = [downloads, lidar, lidar_extract]

lastChance = input("Do you really want to delete? ")

if  "y" in lastChance:
    command = "rmdir /Q /S"
    for d in deleteThis:
        subprocess.run(f'{command} {d}', shell=True, stdout=subprocess.PIPE)
    subprocess.run(f'del /Q /F {root}\\{project}\\stats.csv*', shell=True, stdout=subprocess.PIPE)
    subprocess.run(f'del /Q /F {root}\\{project}\\potree_las_list.txt', shell=True, stdout=subprocess.PIPE)
    
completed = subprocess.run(f'dir {root}\\{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))
print(f"Finished! Copy the {root}\\{project} to your local computer if you are on Virtual Den.")
print(f"Use the zipData.ipynb notebook to compress the {root}\\{project} folder.")
printComplete()

Do you really want to delete? y
 Volume in drive C is Windows
 Volume Serial Number is 6670-36EA

 Directory of c:\SarahsGIS\silo1200

05/10/2021  06:50 PM    <DIR>          .
05/10/2021  06:50 PM    <DIR>          ..
05/10/2021  06:48 PM    <DIR>          lidar_color
05/10/2021  06:50 PM    <DIR>          potree
05/10/2021  06:49 PM    <DIR>          workspace.gdb
               0 File(s)              0 bytes
               5 Dir(s)  781,737,201,664 bytes free

Finished! Copy the c:\SarahsGIS\silo1200 to your local computer if you are on Virtual Den.
Use the zipData.ipynb notebook to compress the c:\SarahsGIS\silo1200 folder.
Cell finished processing 2021-05-10 18:50:52.021179


In [24]:
# Hillshade DEM
# What is the name of the DEM?
dem = 'silo_dem_5ft'

# What are the combinations of azimuth and altitude?
# Azimuth 0 north, 90 east, 180 south, 270 west
azimuth = [180, 270, 315]
# Altitude 0 horizon, 90 directly overhead
altitude = [20, 50, 45]

# Model shadows?
shadows = [True, False, False]

# Vericial exaggeration
vert = [1, 2, 2]

printComplete()

Cell finished processing 2021-05-10 17:55:39.166393
